<a href="https://colab.research.google.com/github/alwaysneedhelp/Practice-for-IOAI/blob/main/skin_cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import kagglehub
import os
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as trans
import torch.nn.functional as F

In [23]:
path = kagglehub.dataset_download('kmader/skin-cancer-mnist-ham10000')
path = os.path.join(path, "hmnist_28_28_L.csv")
df = pd.read_csv(path)


X = df.drop("label", axis=1).values
y = df["label"].values


X = X / 255.0
X = X.reshape(-1, 1, 28, 28)


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [25]:
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                               torch.tensor(y_train, dtype=torch.long))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32),
                             torch.tensor(y_val, dtype=torch.long))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)


In [82]:
class SkinCancer(nn.Module):

  def __init__(self):
    super(SkinCancer, self).__init__()
    self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
    self.pool = nn.MaxPool2d(2)
    self.conv2 = nn.Conv2d(16, 32, 3, padding=1)

    self.fc1 = nn.Linear(32*7*7, 128)
    self.fc2 = nn.Linear(128, 7)

    self.relu = nn.ReLU()

  def forward(self, x):
    x = self.pool(self.relu(self.conv1(x)))
    x = self.pool(self.relu(self.conv2(x)))

    x = torch.flatten(x, 1)
    x = self.relu(self.fc1(x))
    x = (self.fc2(x))

    return x

In [121]:
model = SkinCancer()
loaded = os.path.exists('./models/skin_cancer.pth')
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)

In [122]:
def train(train_loader, loss_function, optimizer, model, epochs=50):

  model.train()

  for epoch in range(epochs):
    print(f'Training {epoch}...')

    total_loss = 0

    for images, labels in train_loader:
      images = images/255
      optimizer.zero_grad()

      outputs = model(images)
      loss = loss_function(outputs, labels)

      total_loss += loss.item()

      loss.backward()
      optimizer.step()

    print(f'Loss: {total_loss/len(train_loader):.4f}')

In [123]:

if not loaded:
  train(
      train_loader,
      loss_function,
      optimizer,
      model
  )
else:
  model.load_state_dict(torch.load('./models/skin_cancer.pth'))

Training 0...
Loss: 1.8524
Training 1...
Loss: 1.7791
Training 2...
Loss: 1.6832
Training 3...
Loss: 1.5559
Training 4...
Loss: 1.4178
Training 5...
Loss: 1.2865
Training 6...
Loss: 1.2083
Training 7...
Loss: 1.1733
Training 8...
Loss: 1.1581
Training 9...
Loss: 1.1493
Training 10...
Loss: 1.1464
Training 11...
Loss: 1.1463
Training 12...
Loss: 1.1416
Training 13...
Loss: 1.1378
Training 14...
Loss: 1.1384
Training 15...
Loss: 1.1385
Training 16...
Loss: 1.1390
Training 17...
Loss: 1.1365
Training 18...
Loss: 1.1378
Training 19...
Loss: 1.1384
Training 20...
Loss: 1.1400
Training 21...
Loss: 1.1388
Training 22...
Loss: 1.1349
Training 23...
Loss: 1.1372
Training 24...
Loss: 1.1381
Training 25...
Loss: 1.1340
Training 26...
Loss: 1.1345
Training 27...
Loss: 1.1377
Training 28...
Loss: 1.1353
Training 29...
Loss: 1.1365
Training 30...
Loss: 1.1367
Training 31...
Loss: 1.1339
Training 32...
Loss: 1.1338
Training 33...
Loss: 1.1400
Training 34...
Loss: 1.1371
Training 35...
Loss: 1.1376
Tr

In [125]:
torch.save(model.state_dict(), 'models/skin_cancer.pth')

In [126]:
def evaluate(test_loader, model, loss_function):
  model.eval()


  correct = 0
  total = 0
  with torch.no_grad():
    for images, labels in test_loader:

      images /= 255

      outputs = model(images)
      loss = loss_function(outputs, labels)
      _, preds = torch.max(outputs, 1)


      total += labels.size(0)
      correct += (preds==labels).sum().item()

  accuracy = 100 * correct/total

  print(f'Accuracy: {accuracy}')

In [127]:
evaluate(
    val_loader,
    model,
    loss_function
)

Accuracy: 68.09785322016974
